<a href="https://colab.research.google.com/github/Noxybot/tt_21/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [202]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras import *

SEED = 228

tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

GEO_CLUSTERS = 300
COMPANY_UNITS = 10
ACTIVATION_FUN = "relu"


In [205]:
train_data = pd.read_csv("train.csv")
train_data = train_data.drop(train_data.columns[0], axis=1)

test_data = pd.read_csv("test.csv")
test_data = test_data.drop(test_data.columns[0], axis=1)

In [197]:
def preprocess (dataframe, kmeans):
  dataframe['geo_cluster_label'] = kmeans.predict(dataframe[dataframe.columns[0:2]])
  dataframe.loc[dataframe["company"].value_counts()[dataframe["company"]].values <= 10, "company"] = -1
  dataframe = dataframe.drop('latitude', axis=1)
  dataframe = dataframe.drop('longitude', axis=1)
  first_X = pd.concat([dataframe["is_local"], dataframe.iloc[:, 3:7]], axis=1).to_numpy()
  second_X = LabelEncoder().fit_transform(dataframe["company"].to_numpy())
  third_X = dataframe["type"].to_numpy()
  fourth_X = dataframe["geo_cluster_label"].to_numpy()

  out_Y = dataframe["target"].to_numpy()
  return first_X, second_X, third_X, fourth_X, out_Y



In [206]:
kmeans = KMeans(n_clusters = GEO_CLUSTERS, init ='k-means++')
kmeans.fit(train_data[train_data.columns[0:2]])
first_X_train, second_X_train, third_X_train, fourth_X_train, out_Y_train = preprocess(train_data, kmeans)
first_X_test, second_X_test, third_X_test, fourth_X_test, out_Y_test = preprocess(test_data, kmeans)


In [207]:
input_first_branch = Input(shape=(5, ), name="first")
first = Dense(32, activation=ACTIVATION_FUN)(input_first_branch)
first = BatchNormalization()(first)
first = Dense(32, activation=ACTIVATION_FUN)(first)
first = BatchNormalization()(first)
first = Model(inputs=input_first_branch, outputs=first)

input_second_branch = Input(shape=(1,), name="second")
second = Embedding(second_X_train.max() + 1, output_dim=128)(input_second_branch)
second = Flatten()(second)
second = Dense(32, activation=ACTIVATION_FUN)(second)
second = Model(inputs=input_second_branch, outputs=second)

input_third_branch = Input(shape=(1,), name="third")
third = Embedding(third_X_train.max() + 1, output_dim=16)(input_third_branch)
third = Flatten()(third)
third = Dense(16, activation=ACTIVATION_FUN)(third)
third = Model(inputs=input_third_branch, outputs=third)


input_fourth_branch = Input(shape=(1,), name="fourth")
fourth = Embedding(fourth_X_train.max() + 1, output_dim=128)(input_fourth_branch)
fourth = Flatten()(fourth)
fourth = Dense(32, activation=ACTIVATION_FUN)(fourth)
fourth = Model(inputs=input_fourth_branch, outputs=fourth)

concat = Concatenate()([first.output, second.output, third.output, fourth.output])

concat = Dense(64, activation=ACTIVATION_FUN)(concat)
concat = BatchNormalization()(concat)
concat = Dense(64, activation=ACTIVATION_FUN)(concat)
concat = BatchNormalization()(concat)
output = Dense(1, name="output", activation=ACTIVATION_FUN)(concat)

my_model = Model(inputs=[first.input, second.input, third.input, fourth.input], outputs=output)
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(epochs=10, batch_size=128, x={"first": first_X_train, "second": second_X_train, "third": third_X_train, "fourth": fourth_X_train }, y=out_Y_train)


Epoch 1/10
1251/1251 [==============================] - 11s 7ms/step - loss: 2.1659 - accuracy: 0.7263
Epoch 2/10
1251/1251 [==============================] - 9s 7ms/step - loss: 1.1323 - accuracy: 0.7125
Epoch 3/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.8137 - accuracy: 0.7261
Epoch 4/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.6828 - accuracy: 0.7354
Epoch 5/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.6239 - accuracy: 0.7548
Epoch 6/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.5374 - accuracy: 0.7684
Epoch 7/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.5347 - accuracy: 0.7644
Epoch 8/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.5011 - accuracy: 0.7706
Epoch 9/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.5150 - accuracy: 0.7685
Epoch 10/10
1251/1251 [==============================] - 9s 7ms/step - loss: 0.60

In [208]:
my_model.summary()

Model: "model_199"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 first (InputLayer)             [(None, 5)]          0           []                               
                                                                                                  
 dense_273 (Dense)              (None, 32)           192         ['first[0][0]']                  
                                                                                                  
 second (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 third (InputLayer)             [(None, 1)]          0           []                               
                                                                                          

In [209]:
my_model.evaluate(x={"first": first_X_test, "second": second_X_test, "third": third_X_test, "fourth": fourth_X_test }, y=out_Y_test)


1250/1250 [==============================] - 3s 2ms/step - loss: 0.4920 - accuracy: 0.7667


[0.49202656745910645, 0.7666749954223633]